In [2]:
from langchain_community.document_loaders import TextLoader


In [4]:
from pydantic import BaseModel, ConfigDict, validator
from langchain_core.vectorstores.base import VectorStore


In [6]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [8]:
import langchain_community
langchain_community.__version__

'0.3.5'

In [9]:
import langchain
langchain.__version__


'0.3.7'

In [11]:
import pydantic 
pydantic.__version__

'2.9.2'

In [13]:
import os 
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 5


In [15]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('ThinkPad_E16_Gen_1_Intel_Spec (1).pdf')
docs=loader.load()

In [17]:
docs

[Document(metadata={'source': 'ThinkPad_E16_Gen_1_Intel_Spec (1).pdf', 'page': 0}, page_content='OVERVIEW\n1.USB 2.0 5.Thunderbolt 4\n2.Ethernet (RJ-45) 6.USB 3.2 Gen 1 (Always On)\n3.Kensington Nano Security Slot 7.HDMI\n4.USB-C 3.2 Gen 2 8.Headphone / microphone combo jack (3.5mm)ThinkPad E16 Gen 1 (Intel)PSREF\nProduct Specifications\nReference\n1 of 10 ThinkPad E16 Gen 1 (Intel) - January 26 2024'),
 Document(metadata={'source': 'ThinkPad_E16_Gen_1_Intel_Spec (1).pdf', 'page': 1}, page_content='PERFORMANCE\nProcessor\nProcessor Family\n13th Generation Intel® U, P or H Series Core i3 / i5 / i7 Processor\nProcessor **\nProcessor \nNameCores Threads Base Frequency Max Frequency CacheProcessor Graphics\nCore i3-1315U6 (2 P-core + 4 E-\ncore)8P-core 1.2GHz / E-core \n0.9GHzP-core 4.5GHz / E-core \n3.3GHz10MBIntel® UHD \nGraphics\nCore i5-1335U10 (2 P-core + 8 \nE-core)12P-core 1.3GHz / E-core \n0.9GHzP-core 4.6GHz / E-core \n3.4GHz12MBIntel® Iris® Xe \nGraphics [1]\nCore i5-1340P12 (4 P

In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
documents[:5]


[Document(metadata={'source': 'ThinkPad_E16_Gen_1_Intel_Spec (1).pdf', 'page': 0}, page_content='OVERVIEW\n1.USB 2.0 5.Thunderbolt 4\n2.Ethernet (RJ-45) 6.USB 3.2 Gen 1 (Always On)\n3.Kensington Nano Security Slot 7.HDMI\n4.USB-C 3.2 Gen 2 8.Headphone / microphone combo jack (3.5mm)ThinkPad E16 Gen 1 (Intel)PSREF\nProduct Specifications\nReference\n1 of 10 ThinkPad E16 Gen 1 (Intel) - January 26 2024'),
 Document(metadata={'source': 'ThinkPad_E16_Gen_1_Intel_Spec (1).pdf', 'page': 1}, page_content='PERFORMANCE\nProcessor\nProcessor Family\n13th Generation Intel® U, P or H Series Core i3 / i5 / i7 Processor\nProcessor **\nProcessor \nNameCores Threads Base Frequency Max Frequency CacheProcessor Graphics\nCore i3-1315U6 (2 P-core + 4 E-\ncore)8P-core 1.2GHz / E-core \n0.9GHzP-core 4.5GHz / E-core \n3.3GHz10MBIntel® UHD \nGraphics\nCore i5-1335U10 (2 P-core + 8 \nE-core)12P-core 1.3GHz / E-core \n0.9GHzP-core 4.6GHz / E-core \n3.4GHz12MBIntel® Iris® Xe \nGraphics [1]\nCore i5-1340P12 (4 P

In [21]:
db = FAISS.from_documents(documents[:15], OllamaEmbeddings())

In [22]:
query = "Give me information about thinkpad bluetooth"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

Refer to Microsoft® link  for more information about the Windows® Performance power slider.
Power Adapter
Power Adapter **[1]
65W USB-C® (2-pin) AC adapter, supports PD 3.0, 100-240V, 50-60Hz•
65W USB-C® (3-pin) AC adapter, supports PD 3.0, 100-240V, 50-60Hz•
Notes:
[1] AC adapter offerings depend on the country.
DESIGN
Display
Display**[1]
Aspect Contrast Color Refresh Viewing SizeResolution Touch TypeBrightness Surface Key FeaturesThinkPad E16 Gen 1 (Intel)PSREF
Product Specifications
Reference
4 of 10 ThinkPad E16 Gen 1 (Intel) - January 26 2024


In [23]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

C:\Users\AISHWARYA\AppData\Local\Temp\ipykernel_8836\2277366689.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm=Ollama(model="llama2")


Ollama()

In [24]:
#Design Chatprompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")


In [25]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [26]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E0D1CBAB50>, search_kwargs={})

In [27]:
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [28]:
response=retrieval_chain.invoke({"input":"ThinkPad E16 Gen 1 Camera"})
response['answer']

'Based on the provided context, the answer to the question "ThinkPad E16 Gen 1 Camera" is:\n\nThe ThinkPad E16 Gen 1 comes with a MIPI computer vision camera, which is located on the front of the laptop. This camera is used for various purposes such as facial recognition, video conferencing, and other imaging tasks. The camera is integrated into the laptop\'s design and does not have an external socket or port for connecting an additional webcam.\n\nThe MIPI computer vision camera in the ThinkPad E16 Gen 1 supports various features such as:\n\n* Facial recognition: The camera can be used to support facial recognition features, allowing users to log in to their laptop with their face.\n* Video conferencing: The camera can be used for video conferencing and other imaging tasks, providing a more immersive experience for remote meetings and collaborations.\n* Other imaging tasks: The camera can also be used for other imaging tasks such as capturing images or videos, and can be integrated w